In [29]:
from datasets import load_dataset
import cohere

In [ ]:
DATASET = "LeoZotos/bio_full"

TEXT_COL = 'Only_Options' # Only_Options or Question_With_Options
EMBEDDING_COL_NAME = 'Emb_Only_Choices'


In [ ]:
cohere_api_key = ""
hf_api_key = ""
with open("../tokens/COHERE.txt", "r") as f:
    cohere_api_key = f.read().strip()
with open("../tokens/HF_TOKEN.txt", "r") as f:
    hf_api_key = f.read().strip()
co = cohere.Client(cohere_api_key)

data = load_dataset(DATASET, split="train", token=hf_api_key )

In [ ]:
def get_embeddings(batch):
    response = co.embed(
        # texts=batch['Question_With_Options'],
        texts=batch[TEXT_COL],
        model='embed-multilingual-v3.0',
        input_type='search_query' # it is the query, not the document
    )
    return {EMBEDDING_COL_NAME: response.embeddings}

data_with_embeddings = data.map(
    get_embeddings,
    batched=True,
    batch_size=96 
)

print(data_with_embeddings)

Map: 100%|██████████| 667/667 [00:04<00:00, 134.00 examples/s]

Dataset({
    features: ['id', 'Question', 'Answer_A', 'Answer_B', 'Answer_C', 'Answer_D', 'Answer_E', 'Answer_F', 'Answer_G', 'Answer_H', 'Answer_I', 'Answer_J', 'Answer_Key', 'Answer_Text', 'Question_With_Options', 'Correct_Answer_Rate', 'emb', 'Relevant_Docs_en', 'only_choices_text', 'emb_only_choices'],
    num_rows: 667
})


In [ ]:
# Sanity Check: print the first embedding
print(data_with_embeddings[0][EMBEDDING_COL_NAME])

[0.03375244, 0.05227661, 0.013687134, 0.023498535, 0.021087646, 0.04623413, -0.0029010773, -0.061950684, -0.023651123, -0.0017089844, 0.0340271, -0.02607727, 0.005695343, -0.0066566467, -0.0121536255, 0.004306793, 0.062927246, 0.010887146, 0.05987549, -0.05065918, -0.0647583, 0.010139465, 0.09442139, -0.054351807, 0.026138306, 0.022735596, 0.03741455, -0.004673004, 0.07757568, 0.006095886, 0.009544373, -0.050628662, 0.07458496, -0.0003452301, 0.0034484863, -0.011917114, 0.024993896, -0.010147095, 0.04949951, -0.0206604, -0.00642395, -0.013687134, -0.026855469, 0.04800415, 0.0048294067, -0.034851074, 0.025878906, 0.03326416, 0.003643036, 0.02305603, -0.0031814575, -0.016204834, 0.0067749023, -0.0029888153, 0.03967285, 0.0040512085, 0.0104599, -0.00447464, 0.0053367615, -0.013000488, 0.0047340393, -0.008651733, -0.002128601, 0.022506714, 0.043701172, 0.029541016, 0.06390381, 0.027816772, 0.025939941, 0.0054512024, 0.023910522, 0.039245605, 0.015075684, 0.01626587, -0.021636963, -0.006423

In [36]:
data_with_embeddings.push_to_hub(
    repo_id=DATASET,
    commit_message="Add Cohere v3 embeddings to 'emb' column",
    token=hf_api_key,
    private=True
)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.41s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/LeoZotos/usmle_full/commit/6497f820985f87a276f8ed4c3404792f0618774e', commit_message="Add Cohere v3 embeddings to 'emb' column", commit_description='', oid='6497f820985f87a276f8ed4c3404792f0618774e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/LeoZotos/usmle_full', endpoint='https://huggingface.co', repo_type='dataset', repo_id='LeoZotos/usmle_full'), pr_revision=None, pr_num=None)